# Tokenization & Token Counting for Transformer NLP
**Mission:** Build intuition for how raw text becomes token IDs, how counts affect attention cost, and how to reason about efficiency + limits. Each concept: explanation then a minimal code probe.

## 1. What is Tokenization (and why models need discrete units)
Transformers only operate on integer token IDs; these index embedding rows that become vectors the model can mix via attention. Tokenization bridges *text* → *IDs* → *embeddings* → *attention matrices*. Without stable, consistent token boundaries, similarity, caching, cost, and context limits become unpredictable.

In [ ]:
sample_text = 'Transformers compress meaning.'
print('Raw text:', sample_text)
print('Characters:', len(sample_text))

## 2. Raw Text → Simple Word Tokens
Naïve split-based tokenization: fast but brittle (punctuation glued on, case differences create separate tokens, OOV not handled). Shows the *baseline* from which better methods improve.

In [ ]:
words = sample_text.split()
print('Word tokens:', words)
print('Token count (word-level naive):', len(words))

## 3. Character-Level Tokenization (contrast)
Character tokenization guarantees coverage (no OOV) but inflates length → quadratic attention cost skyrockets; also weak semantic grouping.

In [ ]:
chars = list(sample_text)
print('First 20 char tokens:', chars[:20])
print('Total char tokens:', len(chars))

## 4. Subword Tokenization Motivation
Subword schemes (BPE / WordPiece) balance vocab size with ability to compose rare words. Idea: start from characters, iteratively merge frequent pairs to form a compact vocabulary that still decomposes unknown words. This reduces OOV while controlling sequence length vs. character-level.

In [ ]:
# Toy illustration: naive pair frequency count (not a full BPE implementation).
from collections import Counter
word = 'compression'
pairs = [word[i:i+2] for i in range(len(word)-1)]
print('Word:', word)
print('Adjacent bigrams:', pairs)
print('Frequency:', Counter(pairs))

## 5. Using a Pretrained Tokenizer
We load a real WordPiece tokenizer to see how it segments text into subwords and maps them to IDs (embedding indices).

In [ ]:
from transformers import AutoTokenizer
MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokens = tokenizer.tokenize(sample_text)
ids = tokenizer.convert_tokens_to_ids(tokens)
print('Model:', MODEL_NAME)
print('Subword tokens:', tokens)
print('Token IDs:', ids)

## 6. Special Tokens & IDs
Special tokens add structure (classification heads, separation, padding, masking). They consume context length and affect counting.

In [ ]:
print('Special tokens map:')
for k,v in tokenizer.special_tokens_map.items():
    print(f'  {k}: {v}')
print('All special token IDs:', tokenizer.all_special_ids)

## 7. Encoding to Model Inputs
Encoding adds special tokens and produces attention masks (1=keep, 0=ignore). These arrays are the direct model inputs.

In [ ]:
encoded = tokenizer(sample_text, return_tensors='pt')
print('input_ids shape:', encoded['input_ids'].shape)
print('attention_mask shape:', encoded['attention_mask'].shape)
print('input_ids:', encoded['input_ids'][0].tolist())
print('attention_mask:', encoded['attention_mask'][0].tolist())

## 8. Token Counting Nuances
Raw subword count vs. encoded length (includes special tokens). This matters for API limits & memory planning.

In [ ]:
subword_len = len(tokens)
encoded_len = encoded['input_ids'].shape[1]
print('Subword token count (no specials):', subword_len)
print('Encoded sequence length (with specials):', encoded_len)

## 9. Truncation & Padding
Controlling length: truncate long inputs, pad shorter to batch uniformly. Attention mask prevents padded positions from influencing outputs.

In [ ]:
long_text = ' '.join(['Transformers']*40)
enc_trunc = tokenizer(long_text, max_length=20, truncation=True, padding='max_length', return_tensors='pt')
print('Original repeated tokens length (approx raw word count):', len(long_text.split()))
print('Truncated+Padded input_ids length:', enc_trunc['input_ids'].shape[1])
print('Attention mask:', enc_trunc['attention_mask'][0].tolist())

## 10. Batch Tokenization & Simple Length Distribution
Batch processing shows variability in token counts; plan for worst-case to avoid overflow. We'll render an ASCII histogram (no plotting dependency).

In [ ]:
# FIXED: Batch Tokenization & ASCII length distribution
ADDITIONAL_SENTENCES = [
    'Short prompt.',
    'A surprisingly elongated formulation that still conveys a simple idea.',
    'Numbers like 123456789012345 can behave oddly.',
    'Emojis 🔥🚀 add tokenization quirks.',
    'Hyphenated-terms and URLs https://example.com matter.'
]
batch_enc = tokenizer(ADDITIONAL_SENTENCES, padding=False)
counts = [len(ids) for ids in batch_enc['input_ids']]
print('Token counts per sentence:', counts)
print('ASCII length bars:')
for sent, c in zip(ADDITIONAL_SENTENCES, counts):
    bar = '#' * c
    print(f'{c:3d} | {bar} | {sent[:40]}...')

## 11. Comparing Two Tokenizers
Different vocab segmentation strategies change counts → impacts cost & max context usage. Compare WordPiece vs GPT-2 BPE.

In [ ]:
ALT_MODEL_NAME = 'gpt2'
tok_alt = AutoTokenizer.from_pretrained(ALT_MODEL_NAME)
for text in [sample_text] + ADDITIONAL_SENTENCES[:2]:
    c_main = len(tokenizer.tokenize(text))
    c_alt = len(tok_alt.tokenize(text))
    print(f'TEXT: {text[:40]}...')
    print(f'  {MODEL_NAME} tokens: {c_main}')
    print(f'  {ALT_MODEL_NAME} tokens: {c_alt}')
    print(f'  Difference: {c_alt - c_main}')
    print('-')

## 12. Estimating Attention Cost
Self-attention memory/time ~ O(n^2). Doubling tokens ~4x attention matrix size. Provide a helper for relative cost ratios.

In [ ]:
def attention_quadratic_cost(n):
    return n * n
lengths = [len(t) for t in [tokens, tokenizer.tokenize(long_text)]]
small, large = lengths[0], lengths[1]
ratio = attention_quadratic_cost(large)/attention_quadratic_cost(small)
print(f'Small length: {small}, Large length: {large}')
print(f'Relative O(n^2) cost factor ≈ {ratio:.1f}x')

## 13. Practical Utilities
Reusable helpers for counting, cost estimation (e.g., API pricing per 1K tokens), and safe truncation.

In [ ]:
PRICE_PER_1K = 0.003  # example placeholder
def count_tokens(text, tok):
    return len(tok.tokenize(text))
def estimate_cost(num_tokens, price_per_1k=PRICE_PER_1K):
    return (num_tokens/1000.0)*price_per_1k
def truncate_to_limit(text, tok, limit):
    ids = tok(text)['input_ids']
    if len(ids) <= limit: return text
    # naive: progressively chop words until under limit
    words = text.split()
    while words and len(tok(' '.join(words))['input_ids']) > limit:
        words.pop()
    return ' '.join(words)
num = count_tokens(sample_text, tokenizer)
print('Tokens:', num)
print('Estimated cost ($):', round(estimate_cost(num), 6))
print('Truncated sample (limit 5 tokens):', truncate_to_limit(long_text, tokenizer, 5))

## 14. Edge Cases
Edge forms stress tokenizers: emojis, multilingual text, long numeric strings, URLs. Inspect counts to foresee worst-case consumption.

In [ ]:
edge_texts = {
  'Emojis': 'I love embeddings 😄🔥🚀',
  'Mixed Lang': 'English 与 中文 mixed together.',
  'Long Number': 'Transaction ID 123456789012345678901234567890',
  'URL': 'Check https://sub.domain.example/path?query=1',
  'Code': 'def tokenize(x): return x.split()'
}
for label, txt in edge_texts.items():
    c = count_tokens(txt, tokenizer)
    print(f'{label:11s} | tokens={c:3d} | {txt[:50]}...')

## 15. Mini Exercises
1. Find a sentence that produces >2x more word-level tokens than subword tokens.
2. Rewrite a verbose prompt to reduce its subword token count by ≥20% without losing intent.
3. Construct an example where GPT-2 tokenizer yields significantly more tokens than BERT (why?).
4. Estimate cost difference between two prompts given a 4K context limit.
5. Add a function to cache tokenized results to avoid recomputation.

## 16. Summary (Methods vs Trade-offs)
| Method | Pros | Cons | Typical Use |
|--------|------|------|------------|
| Word (split) | Simple | OOV, punctuation issues | Quick baselines |
| Char | No OOV | Very long sequences | Specialized scripts, OCR |
| Subword (BPE/WordPiece) | Balance length + coverage | Breaks some words into pieces | Modern Transformers |
| SentencePiece (Unigram) | Language-agnostic, robust | Slightly slower | Multilingual models |

**Key heuristics:** Track token counts early; optimize prompts / inputs by reducing redundancy; choose tokenizer consistent with downstream model ecosystem. Attention cost grows *quadratically* with sequence length—every token matters.